In [1]:
import pandas as pd
from datetime import datetime


df = pd.read_csv('linkedin_job_listings_enhanced.csv')

df['company_name'] = df['company_name'].str.replace(r'hiring.*', '', regex=True).str.strip()


df['job_title'] = df['job_title'].str.replace(r' - .*| in .*', '', regex=True).str.strip()


department_mapping = {
    'Product': 'Product',
    'Marketing': 'Marketing',
    'Engineering': 'Engineering',
    'Design': 'Design',
    'Data': 'Data',
    'Sales': 'Sales',
    'CSM': 'Customer Success',
    'HR': 'Human Resources',
    'Other': 'Other'
}
df['department'] = df['department'].replace(department_mapping)


df['location'] = df['location'].str.extract(r'(?:in|at)\s([\w\s,]+)')[0].fillna('Remote').str.strip()


seniority_mapping = {
    'Mid': 'Mid-level',
    'Senior': 'Senior',
    'Lead': 'Lead',
    'Director': 'Director',
    'VP': 'Vice President',
    'Junior': 'Junior',
    'Intern': 'Intern'
}
df['seniority'] = df['seniority'].replace(seniority_mapping)


df['timestamp'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

In [2]:

def categorize_role(title):
    title = title.lower()
    if 'product manager' in title or 'product marketing' in title:
        return 'Product Management'
    elif 'engineer' in title or 'developer' in title:
        return 'Engineering'
    elif 'designer' in title or 'ux' in title or 'ui' in title:
        return 'Design'
    elif 'data scientist' in title or 'data analyst' in title:
        return 'Data Science'
    elif 'sales' in title or 'account executive' in title:
        return 'Sales'
    elif 'customer success' in title:
        return 'Customer Success'
    elif 'recruiter' in title or 'talent' in title:
        return 'Talent Acquisition'
    elif 'marketing' in title:
        return 'Marketing'
    else:
        return 'Other'

df['role_function'] = df['job_title'].apply(categorize_role)

In [3]:
tags = ['Leadership Upgrade', 'New Product Initiative', 'Engineering Buildout', 
        'Design Overhaul', 'Go-To-Market Expansion', 'General Hiring']

for tag in tags:
    df[tag] = df['growth_signal_tag'].str.contains(tag).astype(int)

df.drop('growth_signal_tag', axis=1, inplace=True)

In [4]:
print(df.isnull().sum())

df = df.drop_duplicates(subset=['job_url'])

text_cols = ['company_name', 'job_title', 'department', 'location']
df[text_cols] = df[text_cols].apply(lambda x: x.str.title())

company_name              0
job_title                 0
department                0
location                  0
seniority                 0
job_url                   0
timestamp                 0
role_function             0
Leadership Upgrade        0
New Product Initiative    0
Engineering Buildout      0
Design Overhaul           0
Go-To-Market Expansion    0
General Hiring            0
dtype: int64


In [5]:
df.head()

,company_name,job_title,department,location,seniority,job_url,timestamp,role_function,Leadership Upgrade,New Product Initiative,Engineering Buildout,Design Overhaul,Go-To-Market Expansion,General Hiring
0,Chargebee,Chargebee Hiring Product Marketing Manager,Marketing,Remote,Mid-level,https://www.linkedin.com/jobs/view/product-mar...,2025-06-21 20:50:47,Product Management,1,1,0,0,0,0
1,Chargebee,Chargebee Hiring Staff Product Manager,Product,Remote,Mid-level,https://in.linkedin.com/jobs/view/staff-produc...,2025-06-21 20:50:47,Product Management,1,1,0,0,0,0
2,Chargebee,Large Enterprise Program Manager,Other,Remote,Mid-level,https://www.linkedin.com/jobs/view/large-enter...,2025-06-21 20:50:47,Other,1,0,0,0,0,0
3,Chargebee,Product Marketing Manager -Saas,Marketing,Remote,Mid-level,https://in.linkedin.com/jobs/view/product-mark...,2025-06-21 20:50:47,Product Management,1,1,0,0,0,0
4,Chargebee,Product Marketing Manager,Marketing,Remote,Mid-level,https://in.linkedin.com/jobs/view/product-mark...,2025-06-21 20:50:47,Product Management,1,1,0,0,0,0


In [6]:
import pandas as pd
import re

def clean_job_title(title):
   
    title = re.sub(r'^.*?hiring\s*', '', title, flags=re.IGNORECASE)
    
   
    title = re.sub(r'\s*(?:in|at|for|on|from)\s+.*$', '', title, flags=re.IGNORECASE)
    
    title = re.sub(r'[-–—]\s*$', '', title)

    title = re.sub(r'[.,;]\s*$', '', title)
    
    title = re.sub(r'\(.*?\)', '', title)
    
    title = title.strip()
    
    title = title.title()
    
    return title


df['job_title'] = df['job_title'].apply(clean_job_title)

In [7]:
def clean_location(location):
    
    location = re.sub(r'\(?\s*Remote\s*\)?', '', location, flags=re.IGNORECASE)
    
   
    location = re.sub(r'\s*,.+$', '', location)
    
    
    location = re.sub(r'\s+(?:USA|US|UK|India|Canada|EMEA|APAC|NA|EMEA|APJ)$', '', location, flags=re.IGNORECASE)
    
    
    location = re.sub(r'[^a-zA-Z\s]', '', location)
    location = location.strip()
    
    
    location_mapping = {
        'Bengaluru': 'Bangalore',
        'Banglore': 'Bangalore',
        'Gurugram': 'Gurgaon',
        'NCR': 'Delhi NCR',
        'NAMER': 'North America',
        'EMEA': 'Europe/Middle East/Africa'
    }
    
    location = location_mapping.get(location, location)
    
    
    if not location:
        return 'Remote'
    
    return location.title()


df['location'] = df['location'].apply(clean_location)

In [8]:
print(df[['job_title', 'location']].head(10))

                                        job_title  \
0                       Product Marketing Manager   
1                           Staff Product Manager   
2                Large Enterprise Program Manager   
3                 Product Marketing Manager -Saas   
4                       Product Marketing Manager   
5             Business Development Representative   
6             Business Development Representative   
7        Business Development Representative Emea   
8  Enterprise Business Development Representative   
9                    Enterprise Account Executive   

                                            location  
0                                             Remote  
1                                             Remote  
2                                             Remote  
3                                             Remote  
4                                             Remote  
5                                             Remote  
6  Empowers Businesses To Strea

In [9]:
print("\nUnique job titles:", df['job_title'].nunique())
print("Unique locations:", df['location'].nunique())


Unique job titles: 229
Unique locations: 34


In [10]:
df.head()

,company_name,job_title,department,location,seniority,job_url,timestamp,role_function,Leadership Upgrade,New Product Initiative,Engineering Buildout,Design Overhaul,Go-To-Market Expansion,General Hiring
0,Chargebee,Product Marketing Manager,Marketing,Remote,Mid-level,https://www.linkedin.com/jobs/view/product-mar...,2025-06-21 20:50:47,Product Management,1,1,0,0,0,0
1,Chargebee,Staff Product Manager,Product,Remote,Mid-level,https://in.linkedin.com/jobs/view/staff-produc...,2025-06-21 20:50:47,Product Management,1,1,0,0,0,0
2,Chargebee,Large Enterprise Program Manager,Other,Remote,Mid-level,https://www.linkedin.com/jobs/view/large-enter...,2025-06-21 20:50:47,Other,1,0,0,0,0,0
3,Chargebee,Product Marketing Manager -Saas,Marketing,Remote,Mid-level,https://in.linkedin.com/jobs/view/product-mark...,2025-06-21 20:50:47,Product Management,1,1,0,0,0,0
4,Chargebee,Product Marketing Manager,Marketing,Remote,Mid-level,https://in.linkedin.com/jobs/view/product-mark...,2025-06-21 20:50:47,Product Management,1,1,0,0,0,0


In [11]:
df.to_csv('extra cleaned columns.csv', index=False)